In [20]:
import pandas as pd

In [21]:
df = pd.read_csv('/Users/galbraithelroy/Documents/jse-data-extractor/csvs/multiyear_batch/metrics_consolidated - Copy of Sheet1.csv')
df_cleaned = df.copy()

In [22]:
df_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3836 entries, 0 to 3835
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Unnamed: 0       3836 non-null   object 
 1   symbol           3836 non-null   object 
 2   year             3836 non-null   int64  
 3   item_name        3836 non-null   object 
 4   item_type        3781 non-null   object 
 5   item             3836 non-null   object 
 6   standard_item    3682 non-null   object 
 7   unit_multiplier  3827 non-null   float64
 8   confidence       2677 non-null   object 
 9   drive_path       2660 non-null   object 
dtypes: float64(1), int64(1), object(8)
memory usage: 299.8+ KB


In [23]:
df_cleaned.rename(columns={'Unnamed: 0': 'company'}, inplace=True)

In [24]:
df_cleaned.loc[df_cleaned['symbol'] == 'BIL', 'company'] = "Barita Investments Limited"
company_name_mapping = {
    'Caribbean Flavours & Fragrances Limited': 'Caribbean Flavours and Fragrances Limited'
}
df_cleaned['company'] = df_cleaned['company'].replace(company_name_mapping)
companies = sorted(df_cleaned["company"].unique().tolist())
print(companies)
print(len(companies))

['138 Student Living Jamaica Limited', 'A.S. Bryden and Sons Holdings Limited', 'AMG Packaging & Paper Company Limited', 'Access Financial Services Limited', 'Barita Investments Limited', 'Berger Paints Jamaica Limited', 'Blue Power Group Limited', 'CAC 2000 Limited', 'Cargo Handlers Limited', 'Caribbean Cement Company Limited', 'Caribbean Cream Limited', 'Caribbean Flavours and Fragrances Limited', 'Caribbean Producers (Jamaica) Limited', 'Carreras Limited', 'Derrimon Trading Company Limited', 'Dolla Financial Services Limited', 'Dolphin Cove Limited', 'Elite Diagnostic Limited', 'Everything Fresh Limited', 'Fontana Limited', 'FosRich Company Limited', 'Future Energy Source Company Limited', 'GWEST Corporation Limited', 'Gracekennedy Limited', 'HONEY BUN (1982) LIMITED', 'Indies Pharma Jamaica Limited', 'JFP Limited', 'JMMB Group Limited', 'Jamaica Broilers Group Limited', 'Jamaica Producers Group Limited', 'Jamaican Teas Limited', 'Jetcon Corporation Limited', 'K.L.E Group Limited', 

In [25]:
symbols = sorted(df_cleaned["symbol"].unique().tolist())
print(symbols)
print(len(symbols))

['138SL', '1GS', 'AFS', 'AMG', 'ASBH', 'BIL', 'BPOW', 'BRG', 'CAC', 'CAR', 'CCC', 'CFF', 'CHL', 'CPJ', 'DCOVE', 'DOLLA', 'DTL', 'EFRESH', 'ELITE', 'FESCO', 'FOSRICH', 'FTNA', 'GK', 'GWEST', 'HONBUN', 'INDIES', 'JAMT', 'JBG', 'JETCON', 'JFP', 'JMMBGL', 'JP', 'KEX', 'KEY', 'KLE', 'KPREIT', 'KREMI', 'KW', 'LAB', 'LASD', 'LASF', 'LASM', 'LUMBER', 'MDS', 'MEEG', 'MGL', 'NCBFG', 'OMNI', 'ONE', 'PJAM', 'PJX', 'PROVEN', 'PTL', 'QWI', 'RPL', 'SOS', 'SPURTREE', 'TROPICAL']
58


In [26]:
years = sorted(df_cleaned["year"].unique().tolist())
print(years)
print(len(years))

[2012, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023, 2024]
11


In [27]:
item_names = sorted(df["item_name"].unique().tolist())
print(item_names)
print(len(item_names))

['Admin Expenses', 'Admin Expenses Change', 'Administrative Expenses', 'After Tax Profit', 'Associate Count', 'Associates Change', 'Associates Count', 'CAPEX', 'Capital Expenditure', 'Cash', 'Cash and Cash Equivalents', 'Cash and Deposits', 'Current Asset Ratio', 'Current Assets', 'Current Liabilities', 'Current Ratio', 'Debt to Equity', 'Debt to Equity Ratio', 'Dividend Payout Ratio', 'Dividend payout ratio', 'Dividends', 'Dividends per stock unit (cents)', 'EARNINGS PER STOCK UNIT', 'EBITDA', 'EPS', 'Earnings Per Share', 'Earnings per Share', 'Earnings per Stock Unit', 'Earnings per share', 'Earnings per stock unit (cents)', 'Efficiency Ratio', 'Equity', 'Financing Cash Flow', 'Gross Margin', 'Gross Profit', 'Gross Profit Margin', 'Gross Written Premiums', 'Income Tax Expense', 'Insurance Revenue', 'Interest Coverage Ratio', 'Investing Cash Flow', 'Net Cash Flow', 'Net Interest Income', 'Net Loans', 'Net Loss', 'Net Operating Revenue', 'Net Profit', 'Net Profit Attributable to Owners

In [28]:
# Simple item name standardization to snake_case for Jupyter notebook
# Run this cell to standardize the item names

import re

def to_snake_case(text):
    """
    Convert text to snake_case format.
    """
    if pd.isna(text):
        return text
    
    # Convert to string if not already
    text = str(text).strip()
    
    # Handle special cases and abbreviations
    special_mappings = {
        'CAPEX': 'capital_expenditure',
        'EBITDA': 'ebitda',
        'EPS': 'earnings_per_share',
        'P/E RATIO': 'price_earnings_ratio',
        'EARNINGS PER STOCK UNIT': 'earnings_per_stock_unit',
        'OPERATING PROFIT': 'operating_profit',
        'PROFIT ATTRIBUTABLE TO STOCKHOLDERS': 'profit_attributable_to_stockholders',
        'PROFIT BEFORE TAXATION': 'profit_before_taxation',
        "STOCKHOLDERS' RETURN ON EQUITY": 'stockholders_return_on_equity',
        'Net profit(loss) ': 'net_profit_loss',
        'Profit / (Loss) before tax': 'profit_loss_before_tax',
        'Profit/(loss) before taxation': 'profit_loss_before_taxation',
        'Operating revenue net of interest expense': 'operating_revenue_net_of_interest_expense'
    }
    
    # Check if it's a special case first
    if text in special_mappings:
        return special_mappings[text]
    
    # Replace special characters and normalize spaces
    text = re.sub(r'[^\w\s]', ' ', text)  # Replace non-word chars with spaces
    text = re.sub(r'\s+', ' ', text)      # Normalize multiple spaces to single space
    text = text.strip()
    
    # Convert to lowercase and replace spaces with underscores
    text = text.lower().replace(' ', '_')
    
    # Remove any remaining special characters
    text = re.sub(r'[^a-z0-9_]', '', text)
    
    # Remove leading/trailing underscores
    text = text.strip('_')
    
    return text

# Apply snake_case conversion to item_name column
df_cleaned['item_name_standardized'] = df_cleaned['item_name'].apply(to_snake_case)

# Show examples of the conversion
print("Examples of conversion:")
print("Original -> Standardized")
print("-" * 50)

# Get a sample of conversions to show
sample_items = df_cleaned[['item_name', 'item_name_standardized']].drop_duplicates().head(15)
for _, row in sample_items.iterrows():
    original = row['item_name']
    standardized = row['item_name_standardized']
    print(f"{original} -> {standardized}")

# Show summary
print(f"\nSummary:")
print(f"  - Original unique items: {len(df['item_name'].unique())}")
print(f"  - Standardized unique items: {len(df_cleaned['item_name_standardized'].unique())}")

# Check for duplicates created by standardization
duplicates = df_cleaned.groupby('item_name_standardized').filter(lambda x: len(x) > 1)
if not duplicates.empty:
    print(f"\nPotential duplicates after standardization:")
    for item in duplicates['item_name_standardized'].unique():
        original_items = duplicates[duplicates['item_name_standardized'] == item]['item_name'].tolist()
        print(f"  '{item}': {original_items}")

# Update the original dataframe
print("\nDataframe updated successfully!")

Examples of conversion:
Original -> Standardized
--------------------------------------------------
Operating Profit -> operating_profit
Net Profit -> net_profit
Total Assets -> total_assets
Shareholders Equity -> shareholders_equity
debt_to_equity_ratio -> debt_to_equity_ratio
liquidity_ratio -> liquidity_ratio
Revenue -> revenue
Total Revenue -> total_revenue
Gross Profit -> gross_profit
EBITDA -> ebitda
Return on Equity -> return_on_equity
Shareholders' Equity -> shareholders_equity
Earnings per Share -> earnings_per_share
Gross Profit Margin -> gross_profit_margin
Operating_Profit_Margin -> operating_profit_margin

Summary:
  - Original unique items: 182
  - Standardized unique items: 136

Potential duplicates after standardization:
  'operating_profit': ['Operating Profit', 'Operating Profit', 'Operating Profit', 'Operating Profit', 'Operating Profit', 'Operating Profit', 'Operating Profit', 'Operating Profit', 'Operating Profit', 'Operating Profit', 'Operating Profit', 'Operating

In [29]:
df_cleaned[df_cleaned["item_type"].isna()]["item_name_standardized"].unique().tolist()

['ebitda',
 'earnings_per_share',
 'gross_profit',
 'gross_profit_margin',
 'net_profit',
 'net_profit_margin',
 'operating_profit',
 'return_on_assets',
 'return_on_equity',
 'revenue',
 'total_assets',
 'total_equity',
 'current_ratio',
 'debt_to_equity_ratio']

In [30]:
item_types = (df_cleaned["item_type"].unique().tolist())
print(item_types)
print(len(item_types))

['line_item', 'ratio', 'line item', nan]
4


In [31]:
# Define the standardization mapping for item_type
item_type_standardization = {
    'line item': 'line_item',  # Convert 'line item' to 'line_item'
    'line_item': 'line_item',  # Keep 'line_item' as is
    'ratio': 'ratio',          # Keep 'ratio' as is
}

# Apply the standardization
df_cleaned['item_type'] = df_cleaned['item_type'].replace(item_type_standardization)

In [32]:
# Fill missing item_type values using existing classifications
# This ensures consistency with your current data

# Step 1: Create a mapping dictionary from existing non-null item_type data
print("Creating mapping dictionary from existing classifications...")

# Get all rows where item_type is not null
non_null_types = df_cleaned[df_cleaned['item_type'].notna()][['item_name_standardized', 'item_type']].drop_duplicates()

# Create mapping from standardized item name to item_type
item_type_mapping = {}
for _, row in non_null_types.iterrows():
    standardized_name = row['item_name_standardized']
    item_type = row['item_type']
    
    # If we already have this standardized name mapped, check for consistency
    if standardized_name in item_type_mapping:
        if item_type_mapping[standardized_name] != item_type:
            print(f"WARNING: Inconsistent classification for '{standardized_name}':")
            print(f"  - Previous: {item_type_mapping[standardized_name]}")
            print(f"  - Current: {item_type}")
    else:
        item_type_mapping[standardized_name] = item_type

print(f"\nCreated mapping for {len(item_type_mapping)} unique standardized items")



Creating mapping dictionary from existing classifications...
  - Previous: ratio
  - Current: line_item
  - Previous: ratio
  - Current: line_item
  - Previous: ratio
  - Current: line_item
  - Previous: line_item
  - Current: ratio
  - Previous: line_item
  - Current: ratio
  - Previous: line_item
  - Current: ratio
  - Previous: line_item
  - Current: ratio
  - Previous: ratio
  - Current: line_item
  - Previous: ratio
  - Current: line_item
  - Previous: ratio
  - Current: line_item

Created mapping for 136 unique standardized items


In [33]:
print("\nFilling missing item_type values...")

# Count missing values before
missing_before = df_cleaned['item_type'].isna().sum()
print(f"Missing item_type values before: {missing_before}")

# Fill missing values using the mapping
filled_count = 0
for idx, row in df_cleaned.iterrows():
    if pd.isna(row['item_type']) and row['item_name_standardized'] in item_type_mapping:
        df_cleaned.at[idx, 'item_type_filled'] = item_type_mapping[row['item_name_standardized']]
        filled_count += 1
        
# Fill missing values of item_type_filled with item_type
df_cleaned['item_type_filled'] = df_cleaned['item_type_filled'].fillna(df_cleaned['item_type'])

# Count missing values after
missing_after = df_cleaned['item_type_filled'].isna().sum()
print(f"Missing item_type values after: {missing_after}")
print(f"Filled {filled_count} missing values")


Filling missing item_type values...
Missing item_type values before: 55
Missing item_type values after: 0
Filled 55 missing values


In [34]:
item_types = (df_cleaned["item_type_filled"].unique().tolist())
print(item_types)
print(len(item_types))

['line_item', 'ratio']
2


In [35]:
# rename item as item_value
df_cleaned.rename(columns={'item': 'item_value'}, inplace=True)

# convert string to float
df_cleaned['item_value'] = pd.to_numeric(df_cleaned['item_value'], errors='coerce')

# set item_value to float
df_cleaned['item_value'] = df_cleaned['item_value'].astype(float)

In [36]:
canonical_item_names = df_cleaned["standard_item"].str.lower().unique().tolist()
print(canonical_item_names)
print(len(canonical_item_names))

['operating_profit', 'net_profit', 'total_assets', 'total_equity', 'debt_to_equity_ratio', 'liquidity_ratio', 'revenue', 'gross_profit', 'ebitda', 'roe', 'eps', 'gross_profit_margin', 'operating_margin', 'net_profit_margin', 'profit_before_tax', 'current_ratio', 'none', 'roa', 'total_liabilities', 'effective_tax_rate', 'net_margin', 'return_on_sales', 'dividend_payout_ratio', 'gross_margin', 'p/e', 'dividend_per_share', 'ebitda_margin', 'quick_ratio', 'gross_proft', nan, 'interest_income', 'debt_assets_ratio', 'profit_before_taxation', 'proffit_before_tax', 'overhead_to_revenue', 'net_loss', 'revenue_growth', 'price_to_book', 'asset_turnover_ratio', 'return_on_capital_employed', 'profit_before_tax_to_sales', 'eps ', 'efficiency_ratio']
43


In [37]:
canonical_item_mapping = {
    # Core financial metrics
    'operating_profit': 'operating_profit',
    'operating_earnings': 'operating_profit',  # synonym in US reporting
    'net_profit': 'net_profit',
    'net_income': 'net_profit',
    'net_profit_loss': 'net_profit',
    'net_profit_attributable_to_owners': 'net_profit',
    'net_profit_attributable_to_stockholders': 'net_profit',
    'profit_attributable_to_stockholders': 'net_profit',
    'profit_after_tax': 'net_profit',
    'after_tax_profit': 'net_profit',
    'net_loss': 'net_profit',  # normalize loss under net profit (sign will distinguish)

    'total_assets': 'total_assets',
    'total_liabilities': 'total_liabilities',
    'total_liability': 'total_liabilities',
    
    'shareholders_equity': 'total_equity',
    'stockholders_equity': 'total_equity',
    'equity': 'total_equity',
    'total_shareholders_equity': 'total_equity',
    'total_equity': 'total_equity',

    'revenue': 'revenue',
    'total_revenue': 'revenue',
    'revenues': 'revenue',
    'operating_revenue': 'revenue',
    'net_operating_revenue': 'revenue',
    'total_revenues': 'revenue',
    'operating_revenue_net_of_interest_expense': 'revenue',
    'insurance_revenue': 'revenue',
    'gross_written_premiums': 'revenue',
    'total_operating_income': 'revenue',
    'rental_income': 'revenue',
    
    'gross_profit': 'gross_profit',
    'ebitda': 'ebitda',

    # Profit before tax
    'profit_before_tax': 'profit_before_tax',
    'profit_before_taxation': 'profit_before_tax',
    'profit_loss_before_tax': 'profit_before_tax',
    'profit_loss_before_taxation': 'profit_before_tax',
    'pre_tax_profit_or_loss': 'profit_before_tax',
    'pretax_profit': 'profit_before_tax',

    # Ratios and margins
    'gross_profit_margin': 'gross_profit_margin',
    'ratio_gross_margin': 'gross_profit_margin',
    'ratio_gross_profit_margin': 'gross_profit_margin',

    'operating_profit_margin': 'operating_profit_margin',
    'operating_profit_to_revenue': 'operating_profit_margin',
    'ratio_operating_margin': 'operating_profit_margin',

    'net_profit_margin': 'net_profit_margin',
    'ratio_net_margin': 'net_profit_margin',
    'ratio_profit_before_tax_margin': 'profit_before_tax_margin',
    'ratio_ebitda_margin': 'ebitda_margin',

    'current_ratio': 'current_ratio',
    'ratio_current_ratio': 'current_ratio',
    'current_asset_ratio': 'current_ratio',

    'quick_ratio': 'quick_ratio',
    'ratio_quick_ratio': 'quick_ratio',

    'debt_to_equity_ratio': 'debt_to_equity_ratio',
    'ratio_debt_to_equity': 'debt_to_equity_ratio',
    'debt_to_equity': 'debt_to_equity_ratio',

    'debt_assets_ratio': 'debt_to_assets_ratio',
    'ratio_debt_to_assets': 'debt_to_assets_ratio',

    'return_on_equity': 'roe',
    'return_on_average_equity': 'roe',
    'ratio_return_on_equity': 'roe',
    'stockholders_return_on_equity': 'roe',

    'return_on_assets': 'roa',
    'return_on_average_assets': 'roa',
    'return_on_total_asset': 'roa',
    'return_on_average_total_asset': 'roa',
    'ratio_return_on_assets': 'roa',

    'return_on_sales': 'return_on_sales',
    'ratio_return_on_sales': 'return_on_sales',

    'return_on_capital_employed': 'roce',

    'interest_coverage_ratio': 'interest_coverage_ratio',
    
    'ratio_overheads_to_revenue': 'overhead_to_revenue',
    'ratio_revenue_growth': 'revenue_growth',
    'revenue_growth': 'revenue_growth',

    'price_earnings_ratio': 'p/e',
    'ratio_pe_ratio': 'p/e',

    'earnings_per_share': 'eps',
    'eps': 'eps',
    'ratio_eps': 'eps',
    'ratio_earnings_per_share': 'eps',
    'earnings_per_stock_unit': 'eps',
    'earnings_per_stock_unit_cents': 'eps',

    'dividend_per_share': 'dividend_per_share',
    'dividends_per_stock_unit_cents': 'dividend_per_share',
    'ratio_dividend_per_share': 'dividend_per_share',

    'dividend_payout_ratio': 'dividend_payout_ratio',
    'ratio_dividend_payout_ratio': 'dividend_payout_ratio',
    'ratio_dividend_cover': 'dividend_payout_ratio',

    'effective_tax_rate': 'effective_tax_rate',
    'ratio_effective_tax_rate': 'effective_tax_rate',

    # Cash flow metrics
    'operating_cash_flow': 'operating_cash_flow',
    'investing_cash_flow': 'investing_cash_flow',
    'financing_cash_flow': 'financing_cash_flow',
    'net_cash_flow': 'net_cash_flow',
    'free_cash_flow': 'free_cash_flow',  # useful for valuation

    # Interest income
    'interest_income': 'interest_income',
    'net_interest_income': 'net_interest_income',

    # Items not used as canonical KPIs but important in breakdowns
    'non_current_assets': 'none',
    'current_assets': 'none',
    'non_current_liabilities': 'none',
    'current_liabilities': 'none',
    'net_working_capital': 'none',
    'retained_earnings': 'none',
    'retained_earnings_opening_balance': 'none',
    'retained_earnings_closing_balance': 'none',

    'dividends': 'none',
    'income_tax_expense': 'none',
    'payroll_expenses': 'none',
    'administrative_expenses': 'none',
    'admin_expenses': 'none',
    'admin_expenses_change': 'none',
    'selling_expenses': 'none',
    'selling_expenses_change': 'none',
    'operating_expenses': 'none',
    'operating_expense': 'none',
    'other_operating_expenses': 'none',

    'visitor_count': 'none',
    'visitors': 'none',
    'number_of_associates': 'none',
    'associate_count': 'none',
    'associates_count': 'none',
    'associates_change': 'none',

    'capital_expenditure': 'capital_expenditure',
    'borrowing': 'none',
    'cash': 'none',
    'cash_and_cash_equivalents': 'none',
    'cash_and_deposits': 'none',

    'net_loans': 'none',
    'total_income': 'none',
    'profit': 'none',
    'profit_growth': 'none',
}

df_cleaned["canonical_item_name"] = df_cleaned["standard_item"].replace(canonical_item_mapping)

In [38]:
df_cleaned["canonical_item_name"] = df_cleaned["standard_item"].str.lower().str.strip()

df_cleaned[['item_name_standardized', 'canonical_item_name']].drop_duplicates().to_csv('/Users/galbraithelroy/Documents/jse-data-extractor/csvs/multiyear_batch/canonical_item_name.csv', index=False)

# Cleaned Standardized Items

In [39]:
df_cleaned.head()

,company,symbol,year,item_name,item_type,item_value,standard_item,unit_multiplier,confidence,drive_path,item_name_standardized,item_type_filled,canonical_item_name
0,138 Student Living Jamaica Limited,138SL,2024,Operating Profit,line_item,708105.00,operating_profit,1000.0,1,/content/drive/MyDrive/Annual Reports/138SL/13...,operating_profit,line_item,operating_profit
1,138 Student Living Jamaica Limited,138SL,2024,Net Profit,line_item,344799.00,net_profit,1000.0,1,/content/drive/MyDrive/Annual Reports/138SL/13...,net_profit,line_item,net_profit
2,138 Student Living Jamaica Limited,138SL,2024,Total Assets,line_item,10370877.00,total_assets,1000.0,1,/content/drive/MyDrive/Annual Reports/138SL/13...,total_assets,line_item,total_assets
3,138 Student Living Jamaica Limited,138SL,2024,Shareholders Equity,line_item,6132866.00,total_equity,1000.0,1,/content/drive/MyDrive/Annual Reports/138SL/13...,shareholders_equity,line_item,total_equity
4,138 Student Living Jamaica Limited,138SL,2024,debt_to_equity_ratio,ratio,0.52,debt_to_equity_ratio,1.0,NaN,NaN,debt_to_equity_ratio,ratio,debt_to_equity_ratio


In [40]:
cols_to_keep = [
    "company",
    "symbol",
    "year",
    "item_name_standardized",
    "item_type_filled",
    "canonical_item_name",
    "item_value",
    "unit_multiplier",
    "confidence",
    "drive_path"
    ]

df_final = df_cleaned[cols_to_keep]

df_final.to_csv('/Users/galbraithelroy/Documents/jse-data-extractor/csvs/multiyear_batch/cleaned_standardized_items.csv', index=False)